In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from core import BaseCall,user_msg
from typing import List
class Test(BaseCall):
    colors: List[str]
    
async for plan in Test.call([user_msg('coloblack, white and yellow')],stream=True):
    print(plan)


colors=None
colors=None
colors=[]
colors=[]
colors=['black']
colors=['black']
colors=['black', 'white']
colors=['black', 'white']
colors=['black', 'white', 'yellow']
colors=['black', 'white', 'yellow']


In [3]:
from query_agent import init_agent

In [4]:
from sampling import fetch_joined_data,WeightedSampler


In [5]:
df = await fetch_joined_data()

In [17]:
from query_planner import plan_search_queries,execute_search_plan

In [18]:
query = "who are the vexa users?"

In [19]:
execute_search_plan(plan)

TypeError: execute_search_plan() missing 1 required positional argument: 'plan'

NameError: name 'pd' is not defined

In [12]:

async for plan in plan_search_queries(df, query, stream=True):
    if hasattr(plan, 'strategy') and plan.strategy:
        print(f"\nStrategy: {plan.strategy}")
    if hasattr(plan, 'queries') and plan.queries:
        print("\nQueries:")
        for query in plan.queries:
            print(f"\n- Query: {query.query}")
            print(f"  Rationale: {query.rationale}")
            if query.topic_focus:
                print(f"  Topic: {query.topic_focus}")
            if query.include_speakers:
                print(f"  Include Speakers: {', '.join(query.include_speakers)}")




Strategy: To understand who the Vexa users are, we will break down the search into specific queries focusing on user demographics, user feedback, and user engagement. This will help us gather comprehensive insights about the Vexa user base.

Strategy: To understand who the Vexa users are, we will break down the search into specific queries focusing on user demographics, user feedback, and user engagement. This will help us gather comprehensive insights about the Vexa user base.

Strategy: To understand who the Vexa users are, we will break down the search into specific queries focusing on user demographics, user feedback, and user engagement. This will help us gather comprehensive insights about the Vexa user base.

Strategy: To understand who the Vexa users are, we will break down the search into specific queries focusing on user demographics, user feedback, and user engagement. This will help us gather comprehensive insights about the Vexa user base.

Queries:

- Query: None
  Ratio

In [14]:
plan.model_dump()

{'strategy': 'To understand who the Vexa users are, we will break down the search into specific queries focusing on user demographics, user feedback, and user engagement. This will help us gather comprehensive insights about the Vexa user base.',
 'queries': [{'query': 'Vexa user demographics',
   'rationale': 'To identify the characteristics of Vexa users, such as age, profession, and location.',
   'include_speakers': [],
   'exclude_speakers': [],
   'topic_focus': 'user demographics',
   'topic_type': 'analysis'},
  {'query': 'User feedback on Vexa',
   'rationale': 'To gather insights on user experiences and satisfaction levels with Vexa.',
   'include_speakers': [],
   'exclude_speakers': [],
   'topic_focus': 'user feedback',
   'topic_type': 'feedback'},
  {'query': 'Engagement metrics for Vexa users',
   'rationale': 'To analyze how actively users are engaging with Vexa and what features they utilize the most.',
   'include_speakers': [],
   'exclude_speakers': [],
   'topic_f